In [3]:
import numpy as np
import pandas as pd

In [2]:
np.random.seed(42)

In [66]:
df = pd.read_csv('nlp_data.csv', delimiter='       ', quoting=3)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  if __name__ == '__main__':


In [67]:
df.head()

,label,reviews
0,1,The Da Vinci Code book is just awesome.
1,1,this was the first clive cussler i've ever rea...
2,1,i liked the Da Vinci Code a lot.
3,1,i liked the Da Vinci Code a lot.
4,1,I liked the Da Vinci Code but it ultimatly did...


Split Data into Training and Test Data

In [68]:
from sklearn.model_selection import train_test_split

In [69]:
X_train, X_test, y_train, y_test = train_test_split(
    df['reviews'],
    df['label'],
    test_size=0.2, 
    random_state=42
)

# Build the Tokenizer

In [70]:
from tensorflow.python.keras.preprocessing.text import Tokenizer

In [71]:
top_words = 5000

In [72]:
t = Tokenizer(num_words=top_words) # num_words -> Vocablury size

In [73]:
t.fit_on_texts(X_train.tolist())

# Prepare Training and Test Data

Get the word index for each of the word in the review

In [74]:
X_train = t.texts_to_sequences(X_train.tolist())

In [75]:
X_test = t.texts_to_sequences(X_test.tolist())

How many words in each review?

# Pad Sequences - Important

In [76]:
from tensorflow.python.keras.preprocessing import sequence

In [77]:
max_review_length = 300

In [78]:
X_train = sequence.pad_sequences(X_train,maxlen=max_review_length,padding='post')

In [79]:
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length, padding='post')

In [80]:
embedding_vector_length = 50 # how many numbers per word

# Build The Graph

In [81]:
from tensorflow.python.keras.models import Sequential

In [82]:
from tensorflow.python.keras.layers import Dropout, Dense, Embedding, Flatten

In [83]:
model = Sequential()

Add Embedding layer
 - Embedding Layer Input = Batch_Size * Length of each review

In [84]:
model.add(Embedding(top_words + 1,
                    embedding_vector_length,
                    input_length=max_review_length)
         )

Embedding Layer Output - 
[Batch_Size , Review Length , Embedding_Size]

In [85]:
model.add(Flatten())

In [86]:
model.add(Dense(200,activation='relu'))

In [87]:
model.add(Dropout(0.2))

In [88]:
model.add(Dense(100,activation='relu'))

In [89]:
model.add(Dropout(0.2))

In [90]:
model.add(Dense(60,activation='relu'))

In [91]:
model.add(Dropout(0.2))

In [92]:
model.add(Dense(30,activation='relu'))

In [93]:
model.add(Dropout(0.2))

In [94]:
model.add(Dense(1,activation='sigmoid'))

In [95]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

# Execute the graph

In [96]:
model.fit(X_train,y_train,
          epochs=10,
          batch_size=128,          
          validation_data=(X_test, y_test),
         verbose=2)

Train on 5668 samples, validate on 1418 samples
Epoch 1/10
 - 5s - loss: 0.8243 - acc: 0.5129 - val_loss: 0.6851 - val_acc: 0.5585
Epoch 2/10
 - 4s - loss: 0.6875 - acc: 0.5494 - val_loss: 0.6823 - val_acc: 0.5585
Epoch 3/10
 - 4s - loss: 0.6710 - acc: 0.5653 - val_loss: 0.6268 - val_acc: 0.5585
Epoch 4/10
 - 4s - loss: 0.6235 - acc: 0.5651 - val_loss: 0.6702 - val_acc: 0.5585
Epoch 5/10
 - 5s - loss: 0.5295 - acc: 0.7309 - val_loss: 0.4315 - val_acc: 0.8695
Epoch 6/10
 - 4s - loss: 0.4873 - acc: 0.7858 - val_loss: 0.3834 - val_acc: 0.9323
Epoch 7/10
 - 4s - loss: 0.4324 - acc: 0.8275 - val_loss: 0.3762 - val_acc: 0.9118
Epoch 8/10
 - 4s - loss: 0.4116 - acc: 0.8393 - val_loss: 0.3141 - val_acc: 0.9450
Epoch 9/10
 - 4s - loss: 0.3598 - acc: 0.8582 - val_loss: 0.2628 - val_acc: 0.9379
Epoch 10/10
 - 4s - loss: 0.3289 - acc: 0.8638 - val_loss: 0.3855 - val_acc: 0.8984


In [98]:
pred = model.predict_classes(X_test)
pred

array([[1],
       [0],
       [1],
       ..., 
       [0],
       [1],
       [0]])